CNN Model

In [ ]:
def create_model_CNN(input_dim, embedding_dim, embedding_matrix, pad_len, trainable, n1, k, d=0.0, kr=None, br=None):
    myCNN = Sequential()
    myCNN.add(Embedding(input_dim=input_dim, output_dim=embedding_dim, weights=[embedding_matrix], input_length=pad_len,
                        trainable=trainable))
    myCNN.add(Conv1D(n1, kernel_size=k, kernel_regularizer=kr, bias_regularizer=br, activation='relu'))
    myCNN.add(GlobalMaxPooling1D())
    myCNN.add(Dropout(d))
    myCNN.add(Dense(authors, activation='softmax'))
    myCNN.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])
    return myCNN

LSTM Model

In [ ]:
def create_model_LSTM(input_dim, embedding_dim, embedding_matrix, pad_len, trainable, n1, n2, d=0.0):
    myLSTM = Sequential()
    myLSTM.add(
        Embedding(input_dim=input_dim, output_dim=embedding_dim, weights=[embedding_matrix], input_length=pad_len,
                  trainable=trainable))
    myLSTM.add(Bidirectional(LSTM(n1, dropout=d, return_sequences=True)))
    myLSTM.add(GlobalMaxPooling1D())
    myLSTM.add(Dense(n2, activation='relu'))
    myLSTM.add(Dense(4, activation='softmax'))
    myLSTM.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])
    return myLSTM

BiLSTM Model

In [ ]:
def create_model_biLSTM(input_dim, embedding_dim, embedding_matrix, pad_len, trainable, n1, n2, d=0.0):
    myLSTM = Sequential()
    myLSTM.add(
        Embedding(input_dim=input_dim, output_dim=embedding_dim, weights=[embedding_matrix], input_length=pad_len,
                  trainable=trainable))
    myLSTM.add(LSTM(n1, dropout=d, return_sequences=True))
    myLSTM.add(GlobalMaxPooling1D())
    myLSTM.add(Dense(n2, activation='relu'))
    myLSTM.add(Dense(4, activation='softmax'))
    myLSTM.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])
    return myLSTM

BiLSTM and CNN parallel Model with attention

In [ ]:
class AttentionLayer(Layer):
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
        
    def build(self, attention_input):
        super(AttentionLayer, self).build(attention_input)
        lstm_shape, cnn_shape = attention_input
        # Attention Weights for LSTM
        self.W_in = self.add_weight(shape=(lstm_shape[-1], 1),
                                    initializer='glorot_normal',
                                    trainable=True,
                                    name='input_attention_weights')
        
        # Attention Weights for CNN
        self.W_context = self.add_weight(shape=(cnn_shape[-1], 1),
                                         initializer='glorot_normal',
                                         trainable=True,
                                         name='context_attention_weights')
                                      
        # Attention Bias
        self.b = self.add_weight(shape=(lstm_shape[1], 1),
                                 initializer='glorot_normal',
                                 trainable=True,
                                 name='attention_bias')
        
        
        
    def call(self, attention_input):
        lstm_output, cnn_output = attention_input
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W_in) + 
                              K.dot(cnn_output, self.W_context) +
                              self.b), axis=-1)
        at = K.expand_dims(K.softmax(et), axis=-1)
        attention_output = at * lstm_output
        attention_output = K.sum(attention_output, axis=1)
        return attention_output
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

NameError: ignored

In [ ]:
Input1 = Input(shape=(input_len,))
Embedding_layer1 = Embedding(input_dim=input_dim1, 
                             output_dim=embedding_dim1, 
                             weights=[embedding_matrix1], 
                             trainable=False)(Input1)

# Input 1(BiLSTM):
Lstm = Bidirectional(LSTM(128, dropout=0.3, return_sequences=True))(Embedding_layer1)

# Input 2 (CNN):
Conv1 = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(Embedding_layer1)
Dropout1 = Dropout(0.3)(Conv1)
Conv2 = Conv1D(filters=256, kernel_size=4, activation='relu', padding='same')(Embedding_layer1)
Dropout2 = Dropout(0.3)(Conv2)
Conv3 = Conv1D(filters=256, kernel_size=5, activation='relu', padding='same')(Embedding_layer1)
Dropout3 = Dropout(0.3)(Conv3)
merged = concatenate([Dropout1, Dropout2, Dropout3], axis=1)
max_pool = MaxPooling1D(pool_size=3)(merged)

attention = AttentionLayer()([Lstm, max_pool])
Dense1 = Dense(64, activation='relu')(attention)
Dropout_dense = Dropout(0.25)(Dense1)
outputs = Dense(4, activation='softmax')(Dropout_dense)

classifier = Model(inputs=Input1, outputs=outputs)
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
classifier.summary()

BiLSTM with attention

In [ ]:
class AttentionLayer(Layer):
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
        
    def build(self, input_shape):
        self.W_in = self.add_weight(shape=(input_shape[-1], 1),
                                    initializer='glorot_normal',
                                    trainable=True,
                                    name='input_attention_weights')
       
        self.b = self.add_weight(shape=(input_shape[1], 1),
                                        initializer='zeros',
                                        trainable=True,
                                        name='attention_bias')
        
        super(AttentionLayer, self).build(input_shape)
        
        
    def call(self, lstm_output):
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W_in) +
                              self.b), axis=-1)
        at = K.expand_dims(K.softmax(et), axis=-1)
        attention_output = lstm_output * at
        #attention_output = K.sum(attention_output, axis=1)
        return attention_output
    
    
    def compute_output_shape(self, input_state):
        return (input_state[0], input_state[-1])
    
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

In [ ]:
Input1 = Input(shape=(input_len,))
Embedding_layer1 = Embedding(input_dim=input_dim, 
                             output_dim=embedding_dim, 
                             weights=[embedding_matrix], 
                             trainable=False)(Input1)
LSTM1 = Bidirectional(LSTM(64, dropout=0.3, return_sequences=True))(Embedding_layer1)
Attention = AttentionLayer()(LSTM1)
max_pool = MaxPooling1D(pool_size=5)(Attention)
Dropout1 = Dropout(0.3)(max_pool)
Flat = Flatten()(Dropout1)
Dense1 = Dense(512, activation='relu')(Flat)
Dropout2 = Dropout(0.3)(Dense1)
Dense2 = Dense(64, activation='relu')(Dropout2)
Dropout3 = Dropout(0.25)(Dense2)
outputs = Dense(4, activation='softmax')(Dropout3)

model = Model(inputs=Input1, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Serial BiLSTM and CNN model

In [ ]:
Input1 = Input(shape=(input_len,))
Embedding_layer1 = Embedding(input_dim=input_dim, 
                             output_dim=embedding_dim, 
                             weights=[embedding_matrix], 
                             trainable=False)(Input1)
LSTM1 = Bidirectional(LSTM(64, dropout=0.25, return_sequences=True))(Embedding_layer1)
Conv1 = Conv1D(64, kernel_size=5, activation='relu')(LSTM1)
max_pooling1 = MaxPooling1D(pool_size=3)(Conv1)
Dropout2 = Dropout(0.25)(max_pooling1)
Flatten1 = Flatten()(Dropout2)
Dense1 = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(Flatten1)
Dense2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(Dense1)
output = Dense(4, activation='softmax')(Dense2)

myBiLSTM_CNN = Model(inputs=Input1, outputs=output)
myBiLSTM_CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
myBiLSTM_CNN.summary()

Serial BiLSTM and CNN model with attention

In [ ]:
Input1 = Input(shape=(input_len,))
Embedding_layer1 = Embedding(input_dim=input_dim, 
                             output_dim=embedding_dim, 
                             weights=[embedding_matrix], 
                             trainable=False)(Input1)

LSTM1 = Bidirectional(LSTM(128, dropout=0.25, return_sequences=True))(Embedding_layer1)
Attention = AttentionLayer()(LSTM1)
Conv1 = Conv1D(64, kernel_size=5, activation='relu', padding='same')(Attention)
max_pool = GlobalMaxPooling1D()(Conv1)
Dropout2 = Dropout(0.25)(max_pool)
output = Dense(4, activation='softmax')(Dropout2)

myBiLSTM_CNN_Att = Model(inputs=Input1, outputs=output)
myBiLSTM_CNN_Att.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
myBiLSTM_CNN_Att.summary()